* creates `deriv/wed_df.csv`
- make and save dataframe
- file dependency: `NvSclass_df.csv` and `timing_df_viewing.csv` and `timing_df_recall.csv`

In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import sklearn
import brainiak
import nilearn as nl
from nilearn import image, plotting, input_data

from scipy.spatial import distance

pd.options.display.max_rows = 200

### cols
- sub_num
- wed_id
- onset_view
- onset_recal
- offset_view
- offset_recal

In [2]:
""" onset and offset TR for each state of each wedding """
timing_df_view = pd.read_csv('deriv/view_df.csv',index_col=0)
print('\n',timing_df_view.head())
timing_df_recal = pd.read_csv('deriv/timing_df_recall.csv',index_col=0)
print('\n',timing_df_recal.head())
wedclass_df = pd.read_csv('deriv/NvSclass_df.csv',index_col=0)
print('\n',wedclass_df.head())


    wed_num  vid_str  onset_TR  len_TRs  offset_TR
0        0  vid1a_q        12       17         29
1        0  vid1b_q        29        6         35
2        0   vid2_q        35       11         46
3        0   vid3_q        49       15         64
4        0   vid4_q        67       16         83

    sub_num  wed_id  onset_TR  offset_TR wed_class
0        2      34         0         27       NaN
1        2      29        27         46       NaN
2        2      17        46         63       NaN
3        2      38        63         81       NaN
4        2       6        81        124       NaN

   NorS  wed_id  wed_num  sub_num
0    S      20        0        3
1    N      23        1        3
2    N       6        2        3
3    S      17        3        3
4    S      38        4        3


### init wed_df from timing_recall_df

In [3]:
wed_df = timing_df_recal.copy()
wed_df = wed_df.rename(columns={'onset_TR':'onset_recall2','offset_TR':'offset_recall2'})

### include col wed_view_num 

In [4]:
""" 
use `NvSclass_df.csv` to match wedding_id and wedding_viewing_order
"""
## init col
wed_df.loc[:,'wed_view_num']=int(-99)
## loop over rows
for idx,df_row in wed_df.iterrows():  
  try:
    wed_df.loc[idx,'wed_view_num'] = wedclass_df[(
                    wedclass_df.sub_num == df_row.sub_num) & (
                    wedclass_df.wed_id == df_row.wed_id
                  )].wed_num.values[0]
  except:
    print('err sub',df_row.sub_num)
    continue


err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39


### include cols onset_view and offset_view

In [5]:
"""
from `timing_df_viewing.csv` extract onset and offset of viewing 
"""
wed_df.loc[:,'onset_videos']=-99
wed_df.loc[:,'offset_videos']=-99

for idx,df_row in wed_df.iterrows():
  timing_df_view_wed = timing_df_view[timing_df_view.wed_num == df_row.wed_view_num]
  onset_view = timing_df_view_wed[timing_df_view_wed.vid_str.str[:len('vid1a')]=='vid1a'].onset_TR
  offset_view = timing_df_view_wed[timing_df_view_wed.vid_str.str[:len('vid5')]=='vid5'].offset_TR
  if not len(onset_view):
    print('err sub',df_row.sub_num)
    continue
  wed_df.loc[idx,'onset_videos'] = onset_view.values[0]
  wed_df.loc[idx,'offset_videos'] = offset_view.values[0]

err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39


# include path for each wedding

In [6]:
wed_df.loc[:,'path'] = 'N/A'
for idx,wed_df_row in wed_df.iterrows():
  if wed_df_row.wed_view_num==-99:
    continue
  path_df = pd.read_csv('data/behav/silvy_buckets/sub%iday2/%i_viewingPathAB.csv'%(
    100+wed_df_row.sub_num,wed_df_row.sub_num)).fillna('NA')
  path = path_df.loc[wed_df_row.wed_view_num,'SchemaPath']
  wed_df.loc[idx,'path'] = path

# SAVE

In [7]:
wed_df.to_csv('deriv/wed_df.csv')

In [8]:
wed_df[wed_df.sub_num==3]

,sub_num,wed_id,onset_recall2,offset_recall2,wed_class,wed_view_num,onset_videos,offset_videos,path
12,3,22,0,57,N,5,495,567,NB
13,3,19,57,118,N,10,976,1048,NA
14,3,1,118,155,N,9,879,952,NB
15,3,6,155,201,N,2,207,279,NA
16,3,23,201,245,N,1,111,183,NA
17,3,28,245,317,S,11,1074,1152,SA
18,3,34,317,381,N,6,591,663,NB
19,3,17,381,424,S,3,303,375,SB
20,3,2,424,476,S,7,687,759,SA
21,3,29,476,534,S,8,783,856,SA
